In [ ]:
!nvidia-smi

Wed Dec 23 09:35:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html


     |████████████████████████████████| 55.0MB 57kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 16.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588525 sha256=d335d2fe4045fb2208f465365ee6770e78eadbeb122b363a6f520810daf6a62b
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 14.3MB/s 
     |████████████████████████████████| 757kB 15.2MB/s 
     |████████████████████████████████| 890kB 48.6MB/s 
     |████████████████████████████████| 3.0MB 46.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c79a50ddfe160403afc928916eeaeb2ca83e131de85bba79b7e247a3b910394d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a5

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ul7p4_m6
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ul7p4_m6
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=38c75f1df01a0e9518a6863a01725dc468c859205c3dd6dfcabdcb090c2346fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-ac4tqtp7/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2020-12-23 09:40:53--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2020-12-23 09:40:54--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc355f1874c51c15ceb60106cf0e.dl.dropboxusercontent.com/cd/0/get/BFl-3ICEM9HqPfVHZTCGZzTyaKp1C5AYikP4oYU1-dujfvf9NZHNqHNN7gnS7aWX55qEcfRy3DSnSyIlHn9MrROtiocAstdt51pc4BB5XUb2MtdP7O7yj1lxcqmSocisQxM/file?dl=1# [following]
--2020-12-23 09:40:54--  https://uc355f1874c51c15ceb60106cf0e.dl.dropboxusercontent.com/cd/0/get/BFl-3ICEM9HqPfVHZTCGZzTyaKp1C5AYikP4oYU1-dujfvf9NZHNqHNN7gnS7aWX55qEcfRy3DSnSyIlHn9MrROtioc

In [ ]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.7437667846679688 train acc 0.5
epoch 1 batch id 201 loss 0.4492999017238617 train acc 0.5865982587064676
epoch 1 batch id 401 loss 0.41701704263687134 train acc 0.6920978802992519
epoch 1 batch id 601 loss 0.43676453828811646 train acc 0.7403026206322796
epoch 1 batch id 801 loss 0.392888605594635 train acc 0.7672050561797753
epoch 1 batch id 1001 loss 0.29930949211120605 train acc 0.7835757992007992
epoch 1 batch id 1201 loss 0.256233274936676 train acc 0.7960683805162365
epoch 1 batch id 1401 loss 0.3601740002632141 train acc 0.8047042291220556
epoch 1 batch id 1601 loss 0.3382958769798279 train acc 0.8123536071205496
epoch 1 batch id 1801 loss 0.2625627815723419 train acc 0.8188853414769572
epoch 1 batch id 2001 loss 0.320145845413208 train acc 0.8241894677661169
epoch 1 batch id 2201 loss 0.2726685404777527 train acc 0.8285864379827351

epoch 1 train acc 0.8318757110352674


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8823729219948849


epoch 2 batch id 1 loss 0.5106643438339233 train acc 0.8125
epoch 2 batch id 201 loss 0.2061508297920227 train acc 0.8816853233830846
epoch 2 batch id 401 loss 0.34376123547554016 train acc 0.8831047381546134
epoch 2 batch id 601 loss 0.3483799993991852 train acc 0.8861532861896838
epoch 2 batch id 801 loss 0.36805441975593567 train acc 0.8881671348314607
epoch 2 batch id 1001 loss 0.2705391049385071 train acc 0.8907654845154845
epoch 2 batch id 1201 loss 0.2508748769760132 train acc 0.89365632805995
epoch 2 batch id 1401 loss 0.2223130166530609 train acc 0.8960006245538901
epoch 2 batch id 1601 loss 0.21790464222431183 train acc 0.8983155059337914
epoch 2 batch id 1801 loss 0.1727718561887741 train acc 0.9000815519156025
epoch 2 batch id 2001 loss 0.17788632214069366 train acc 0.9024472138930535
epoch 2 batch id 2201 loss 0.22112147510051727 train acc 0.9037582348932304

epoch 2 train acc 0.905087901734926


...


epoch 3 test acc 0.8944413363171355


epoch 4 batch id 1 loss 0.354548841714859 train acc 0.859375
epoch 4 batch id 201 loss 0.05446098372340202 train acc 0.955068407960199
epoch 4 batch id 401 loss 0.07333746552467346 train acc 0.9556577306733167
epoch 4 batch id 601 loss 0.1999778300523758 train acc 0.9573627287853578
epoch 4 batch id 801 loss 0.1600676327943802 train acc 0.9587624843945068
epoch 4 batch id 1001 loss 0.03733227401971817 train acc 0.9605394605394605
epoch 4 batch id 1201 loss 0.034696612507104874 train acc 0.962023834304746
epoch 4 batch id 1401 loss 0.114571712911129 train acc 0.963006334760885
epoch 4 batch id 1601 loss 0.035571761429309845 train acc 0.9640166302311055
epoch 4 batch id 1801 loss 0.08451725542545319 train acc 0.964854594669628
epoch 4 batch id 2001 loss 0.12864041328430176 train acc 0.9655875187406296
epoch 4 batch id 2201 loss 0.09727511554956436 train acc 0.9661020558836892

epoch 4 train acc 0.966696885665529



epoch 4 test acc 0.8965393222506394


epoch 5 batch id 1 loss 0.3768884241580963 train acc 0.90625
epoch 5 batch id 201 loss 0.03163982555270195 train acc 0.9739583333333334
epoch 5 batch id 401 loss 0.024734191596508026 train acc 0.9759585411471322
epoch 5 batch id 601 loss 0.19787074625492096 train acc 0.9761075291181365
epoch 5 batch id 801 loss 0.029310328885912895 train acc 0.9765527465667915
epoch 5 batch id 1001 loss 0.03114418312907219 train acc 0.9775068681318682
epoch 5 batch id 1201 loss 0.011632658541202545 train acc 0.9778830141548709
epoch 5 batch id 1401 loss 0.01822161301970482 train acc 0.9783302105638829
epoch 5 batch id 1601 loss 0.03969306871294975 train acc 0.9786461586508433
epoch 5 batch id 1801 loss 0.08103004097938538 train acc 0.9790567740144365
epoch 5 batch id 2001 loss 0.10036574304103851 train acc 0.9794868190904548
epoch 5 batch id 2201 loss 0.026239220052957535 train acc 0.9795405497501136

epoch 5 train acc 0.979673279294653



epoch 5 test acc 0.8969988810741688


In [ ]:
from google.colab import drive
isMount=!df |grep /content/drive
if len(isMount) < 1 : drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Model Save
torch.save(model.state_dict(), "/content/drive/MyDrive/myBertModel")
